In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# 1.** Dataset Construction**

In [ ]:
products = pd.read_csv("../input/products.csv")

In [ ]:
products.head()

In [ ]:
departments = pd.read_csv("../input/departments.csv")

In [ ]:
departments.head()

In [ ]:
aisles = pd.read_csv("../input/aisles.csv")

In [ ]:
aisles.head()

In [ ]:
productInfo = pd.merge(products, departments, on='department_id', how='left')
allProductInfo = pd.merge(productInfo, aisles, on='aisle_id', how='left')
allProductInfo.head()

### Order Info

In [ ]:
orders = pd.read_csv("../input/orders.csv")

In [ ]:
orders.head(20)

In [ ]:
order_products_prior = pd.read_csv("../input/order_products__prior.csv")
order_products_prior.head()
### previous items in their basket

In [ ]:
order_products_train = pd.read_csv("../input/order_products__train.csv")
order_products_train.head()
##currently whats in their basket

In [ ]:
orderTrainAndPrior = pd.concat([order_products_prior,order_products_train])
orderTrainAndPrior.head()

2. Dataset Exploration

In [ ]:
train = orders.copy()

In [ ]:
train.describe()

In [ ]:
import seaborn as sns

## Trend of orders throughout a week

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,8))

sns.distplot(train['order_dow'],kde = False)

In [ ]:
### Day 0 and Day 1 has the most number of orders
### Can't really assume what day 0 is

## Trend of orders throughout a day

In [ ]:
plt.figure(figsize=(15,8))

sns.distplot(train['order_hour_of_day'],kde = False)

In [ ]:
## Most orders are bw 10AM and 4PM
### Order count has a drop during lunch time. Makes sense

## Order reordering behaviour of users

In [ ]:
orderGrouped = train.groupby(['days_since_prior_order'], as_index=False).agg("count").copy()
orderGrouped = orderGrouped[['days_since_prior_order', 'order_id']].rename(columns={'order_id':'count'})
orderGrouped.head()

In [ ]:
plt.figure(figsize=(50,8))
sns.catplot(x="days_since_prior_order", y="count",
            markers=["^", "o"], linestyles=["-", "--"],
            kind="point", data=orderGrouped, height=10);

In [ ]:
### The sharp spikes seem to be at the end of each week.
### Maybe make a column call isEndOfWeek. Might correlate

### Also one that says EndofMonth. Sharp Spike there

### Overall a decreasing trend though

In [ ]:
order_products_prior.head(15)

1. 1. ## Trend of adding items to the basket

In [ ]:
orderNumber = order_products_prior.copy().groupby(['add_to_cart_order'], as_index=False).count()
orderNumber.head()

In [ ]:
orderNumberWithReordered = order_products_prior.copy().groupby(['add_to_cart_order', 'reordered'], as_index=False).count()
orderNumberWithReordered.head()

In [ ]:
sns.lmplot(x='add_to_cart_order', y='order_id', hue='reordered', data=orderNumberWithReordered, fit_reg=False, height=13)

In [ ]:
## Am I right in assuming that the first order added to the basket seems more likely to be somthing that was reordered?

## How often do people reorder?

In [ ]:
reorderedCount = orderTrainAndPrior.groupby(['reordered'], as_index=False).count()[['reordered','order_id']].rename(columns={'order_id':'count'})
reorderedCount.head()

In [ ]:
plt.figure(figsize=(20,8))
reorderedCount.plot.bar(x='reordered', y='count', figsize=(15,8))

In [ ]:
## More item were reordered

In [ ]:
## How many products to people usually buy?

In [ ]:
x = orders.copy().groupby('user_id').tail(1)[['user_id','order_number']].groupby('order_number', as_index=False).count()
x.head()

In [ ]:
sns.lmplot(x='order_number', y='user_id', data=x, fit_reg=False, height=13)